In [4]:
import tweepy
import nltk
import re,csv
import string
from nltk.classify import *
import nltk.classify.util

#initialize stopWords
stopWords = []
 
def replaceTwoOrMore(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)

def getStopWordList(stopWordListFileName):
    stopWords=[]
    stopWords.append('URL')
 
    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords

st = open('Sentiment_StopWords.txt', 'r')
stopWords = getStopWordList('Sentiment_StopWords.txt')
  
def getFeatureVector(tweet):
    featureVector = []
    words = tweet.split()
    for w in words:
        #Preprocessing
        w = replaceTwoOrMore(w)
        w = w.strip('\'"?,.')
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        if(w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return featureVector
 
def featureExtraction():
    #Reading Tweets
    inpTweets = csv.reader(open('Sentiment_training_data.txt', 'rt'), delimiter=',', quotechar='|')
    tweets = []
    
    for rowTweet in inpTweets:
        sentiment = rowTweet[0]
        tweet = rowTweet[1]
        featureVector = getFeatureVector(tweet)
        tweets.append((featureVector, sentiment))
    return tweets 

tweets = featureExtraction()

#Classifier 
def get_words_in_tweets(tweets):
    all_words = []
    for (text, sentiment) in tweets:
        all_words.extend(text)
    return all_words

def get_word_features(wordlist):
    
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

word_features = get_word_features(get_words_in_tweets(tweets)) 

def extract_features(tweet):
    settweet = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in settweet)
    return features


training_set = nltk.classify.apply_features(extract_features, tweets[601:5400:])
test_set = nltk.classify.apply_features(extract_features, tweets[:600]+tweets[5400:])

#****** Naive Bayes Classifier******************************************

classifier = nltk.NaiveBayesClassifier.train(training_set)

# Accuracy
accuracy = nltk.classify.accuracy(classifier, training_set) 
print(accuracy) 

total = accuracy * 100 
print('Naive Bayes Accuracy with Training Set: %4.2f' % total) 

# Accuracy Test Set
accuracyTestSet = nltk.classify.accuracy(classifier, test_set) 
print(accuracyTestSet)

totalTest = accuracyTestSet * 100 
print('\nNaive Bayes Accuracy with the Test Set: %4.2f' % totalTest)

print('\nMost Informative features')
print(classifier.show_most_informative_features(n=10))

check = ''
while(check!='exit'):
    inpt = input('\nProvide Input(string) or Write exit for Exit\n')
    inpt=inpt.lower()
    print ('\n')
    if inpt == 'exit':
        print ('Exiting the program')
        check = 'exit'
    else:
        inpt = inpt.lower()
        inpt = inpt.split()
        print ('Predicted Sentiment by Naive Bayes Classifier: ' + classifier.classify(extract_features(inpt)) + '\n')

0.893519483225672
Naive Bayes Accuracy with Training Set: 89.35
0.7583333333333333

Naive Bayes Accuracy with the Test Set: 75.83

Most Informative features
Most Informative Features
      contains(comeback) = True           negati : positi =     88.6 : 1.0
        contains(beside) = True           negati : positi =     67.0 : 1.0
         contains(words) = True           positi : negati =     61.7 : 1.0
      contains(describe) = True           positi : negati =     59.0 : 1.0
      contains(thinking) = True           negati : positi =     41.0 : 1.0
          contains(meet) = True           negati : positi =     39.9 : 1.0
        contains(moment) = True           negati : positi =     29.0 : 1.0
        contains(bihday) = True           positi : negati =     23.3 : 1.0
      contains(giveaway) = True           positi : negati =     23.0 : 1.0
        contains(yoongi) = True           negati : positi =     22.3 : 1.0
None

Provide Input(string) or Write exit for Exit
Nishant is a ver